# Edge Elements using Gridap 

In [ ]:
using Gridap
using GridapODEs
using GridapMakie, GLMakie
using LinearAlgebra
using FileIO
using GridapGmsh
using gmsh

model = GmshDiscreteModel("modelos/cubo_esfera_tet.msh")

Ω = Triangulation(model)
degree = 3
dΩ = Measure(Ω,degree)

B0=1.
A0_x(x)=-(B0/2.)*x[2]
A0_y(x)=(B0/2.)*x[1]

dirichlet_tags=["ext"]
dirichlet_values(x) = VectorValue(A0_x(x),A0_y(x),0.0)


order = 0
reffe = ReferenceFE(nedelec,Float64,order)
V = FESpace(model, reffe, conformity=:HCurl, dirichlet_tags=dirichlet_tags)

U= TrialFESpace(V,dirichlet_values)

f=VectorValue(1.,1.,1.)# #Source

a(u,v) =  ∫( (∇ × v)⋅(∇ × u))*dΩ
b(v) =  ∫(f ⋅v)dΩ 

op = AffineFEOperator(a,b,U,V)

ls = LUSolver()
solver = LinearFESolver(ls)

uh, = solve(solver,op)